In [1]:
import sys,os, pickle
from elftools.elf.elffile import ELFFile
from elftools.elf.segments import Segment
from capstone import *
from capstone.x86 import *
from collections import defaultdict
import posixpath


In [2]:
DATA_PATH = './dfg_data/'

In [3]:
file_name = 'array'

with (open(DATA_PATH+file_name , "rb")) as openfile:
    bb_data , ins_data = pickle.load(openfile)


In [4]:
ins_data

{'start': [[['0x10a4', '0x10ad', '0x10b3', '0x10b6'],
   ['0x10a4',
    '0x10a6',
    '0x10a9',
    '0x10aa',
    '0x10ad',
    '0x10b1',
    '0x10b2',
    '0x10b3',
    '0x10b6',
    '0x10bf'],
   ['0x10b8'],
   ['0x10c5'],
   ['0x10b1'],
   ['0x10c5'],
   ['0x10bf']]],
 'deregister_tm_clones': [[['0x10d0',
    '0x10d7',
    '0x10de',
    '0x10e1',
    '0x10e3',
    '0x10ea',
    '0x10ed',
    '0x10ef',
    '0x10f8'],
   ['0x10ea']]],
 'register_tm_clones': [[['0x1100',
    '0x1107',
    '0x110e',
    '0x1111',
    '0x1114',
    '0x1118',
    '0x111c',
    '0x111f',
    '0x1122',
    '0x1124',
    '0x112b',
    '0x112e',
    '0x1130',
    '0x1138'],
   ['0x112b']]],
 '__do_global_dtors_aux': [[['0x10e3', '0x10ea', '0x10ed'],
   ['0x1144',
    '0x114b',
    '0x114d',
    '0x114e',
    '0x1156',
    '0x1159',
    '0x115b',
    '0x1162',
    '0x1167',
    '0x1173',
    '0x1174',
    '0x1178'],
   ['0x116c'],
   ['0x1064'],
   ['0x10f8'],
   ['0x10ea'],
   ['0x10d0', '0x10d7', '0x10de', '

In [6]:
bb_data

{4256: [4256, 4294],
 4260: [4256, 4294],
 4262: [4256, 4294],
 4265: [4256, 4294],
 4266: [4256, 4294],
 4269: [4256, 4294],
 4273: [4256, 4294],
 4274: [4256, 4294],
 4275: [4256, 4294],
 4278: [4256, 4294],
 4280: [4256, 4294],
 4287: [4256, 4294],
 4293: [4256, 4294],
 4304: [4304, 4323],
 4311: [4304, 4323],
 4318: [4304, 4323],
 4321: [4304, 4323],
 4323: [4323, 4335],
 4330: [4323, 4335],
 4333: [4323, 4335],
 4335: [4335, 4337],
 4344: [4344, 4345],
 4352: [4352, 4388],
 4359: [4352, 4388],
 4366: [4352, 4388],
 4369: [4352, 4388],
 4372: [4352, 4388],
 4376: [4352, 4388],
 4380: [4352, 4388],
 4383: [4352, 4388],
 4386: [4352, 4388],
 4388: [4388, 4400],
 4395: [4388, 4400],
 4398: [4388, 4400],
 4400: [4400, 4402],
 4408: [4408, 4409],
 4416: [4416, 4429],
 4420: [4416, 4429],
 4427: [4416, 4429],
 4429: [4429, 4443],
 4430: [4429, 4443],
 4438: [4429, 4443],
 4441: [4429, 4443],
 4443: [4443, 4455],
 4450: [4443, 4455],
 4455: [4455, 4469],
 4460: [4455, 4469],
 4467: [4455,

In [24]:
bin_path = '/home/nahid/reverse/binaries/array'
md = Cs(CS_ARCH_X86, CS_MODE_64)

with open(bin_path, 'rb') as f:
        
        bin_bytearray = bytearray(f.read())
        elffile = ELFFile(f)
        

for fname, connected_comps in ins_data.items():
    print (fname,connected_comps[0])
    
    print(fname,'__'*10)
    for connected_comp in connected_comps[0]:
        if len(connected_comp)<2:
            continue
        bbs = []
        for addr in connected_comp:
            addr_int = int(addr, 16)
#             if addr_int not in bb_data:
#                 continue
#             print(addr, addr_int,':' , bb_data[addr_int]  )
            bb_inf = bb_data[addr_int]  
            
            if bb_inf not in bbs:
                bbs.append(bb_inf)
        print(bbs)
        
        for bb_entry, bb_end in bbs:
            for inst in md.disasm(bin_bytearray[bb_entry:bb_end], bb_entry):
                print( inst.address , ' ', hex(inst.address) in connected_comp,'    ',hex(inst.address), '  : ' , inst.mnemonic+"  "+inst.op_str   )
                if inst.address == bb_end:
                    break
            
#     break
    

start [['0x10a4', '0x10ad', '0x10b3', '0x10b6'], ['0x10a4', '0x10a6', '0x10a9', '0x10aa', '0x10ad', '0x10b1', '0x10b2', '0x10b3', '0x10b6', '0x10bf'], ['0x10b8'], ['0x10c5'], ['0x10b1'], ['0x10c5'], ['0x10bf']]
start ____________________
[[4256, 4294]]
4256   False      0x10a0   :  endbr64  
4260   True      0x10a4   :  xor  ebp, ebp
4262   False      0x10a6   :  mov  r9, rdx
4265   False      0x10a9   :  pop  rsi
4266   False      0x10aa   :  mov  rdx, rsp
4269   True      0x10ad   :  and  rsp, 0xfffffffffffffff0
4273   False      0x10b1   :  push  rax
4274   False      0x10b2   :  push  rsp
4275   True      0x10b3   :  xor  r8d, r8d
4278   True      0x10b6   :  xor  ecx, ecx
4280   False      0x10b8   :  lea  rdi, [rip + 0xca]
4287   False      0x10bf   :  call  qword ptr [rip + 0x2f13]
4293   False      0x10c5   :  hlt  
[[4256, 4294]]
4256   False      0x10a0   :  endbr64  
4260   True      0x10a4   :  xor  ebp, ebp
4262   True      0x10a6   :  mov  r9, rdx
4265   True      0x10a9 